# 🛒 Capstone Phase 3: Multivariate Modeling (With Exogenous Regressors)

**Goal**: Beat the Univariate baseline by adding context.
**Multivariate** here means: *"Predict Sales using [Sales History, Price, Promo, Day of Week, Month]"*

---

## 1. Setup & Load Full Features

We load the "Tabular" dataset from Notebook 1, which contains all the extra features we ignored in Notebook 2.

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

df = pd.read_parquet('../data/processed/tabular_train_data.parquet')
df['unique_id'] = df['store_id'].astype(str) + '_' + df['item_id'].astype(str)

# Same subset and split as Notebook 2 for fair comparison
top_items = df['unique_id'].unique()[:5]
df_subset = df[df['unique_id'].isin(top_items)]

test_start = df['date'].max() - pd.Timedelta(days=28)
train = df[df['date'] < test_start]
test = df[df['date'] >= test_start]

print(f"Train Shape: {train.shape}")

## 2. Prophet (Multivariate / With Regressors)

We feed `is_promo` and `sell_price` into Prophet. It helps the model understand that a spike was due to a discount, not random noise.

In [ ]:
mae_prophet = []

for uid in top_items:
    item_df = df_subset[df_subset['unique_id'] == uid].copy()
    item_train = item_df[item_df['date'] < test_start].rename(columns={'date': 'ds', 'sales': 'y'})
    item_test = item_df[item_df['date'] >= test_start].rename(columns={'date': 'ds', 'sales': 'y'})
    
    m = Prophet(daily_seasonality=False, weekly_seasonality=True)
    m.add_regressor('is_promo')
    m.add_regressor('sell_price')
    
    m.fit(item_train)
    
    # Forecast needs future regressor values
    # In production, we would know future prices/promos
    future = item_test[['ds', 'is_promo', 'sell_price']]
    
    forecast = m.predict(future)
    preds = forecast['yhat'].values
    
    mae = np.mean(np.abs(item_test['y'].values - preds))
    mae_prophet.append(mae)

print(f"Average MAE (Prophet Multivariate): {np.mean(mae_prophet):.2f}")

## 3. LightGBM (Multivariate / All Features)

Now we give LightGBM everything: Rolling windows, calendar info, prices.
This allows it to learn complex interactions like: *"Sales are high on Mondays ONLY IF there is a promotion."*

In [ ]:
features = [
    'lag_7', 'lag_14', 'lag_28',
    'rolling_mean_7', 'rolling_std_7', 
    'month', 'day_of_week', 'is_weekend', 
    'sell_price', 'is_promo' 
]
target = 'sales'

model = lgb.LGBMRegressor(verbose=-1, seed=42)
model.fit(train[features], train[target])

preds = model.predict(test[features])
mae_lgb = np.mean(np.abs(test[target] - preds))

print(f"Average MAE (LightGBM Multivariate): {mae_lgb:.2f}")

lgb.plot_importance(model, max_num_features=10)
plt.title("Feature Importance: Notice Price/Promo impact")
plt.show()

## 4. Deep Learning: Temporal Fusion Transformer (TFT)

TFT is inherently designed for this. It has specific "buckets" for inputs:
- `static_categoricals`: Store ID (Does this store generally sell more?)
- `time_varying_known_reals`: Price, Month (We know these in the future)
- `time_varying_unknown_reals`: Sales (We only know this in the past)

We use the same PyTorch Forecasting setup as before, but this time we appreciate *why* it's powerful: it aligns all these different data types automatically.

In [ ]:
# (Code identical to previous Advanced Notebook for TFT, but conceptually fits here)
# We skip re-pasting the massive PyTorch block to keep the notebook clean, 
# but in the final project, TFT belongs in this 'Multivariate' category.